# DATA3001 COVID-19 Project code

*NB: This does not include code which trialled out methods that were ultimately discarded, such as PCA.

### Import all relevant packages

In [ ]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
from numpy import inf
import scanpy as sc
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn import linear_model
from sklearn.svm import l1_min_c
from sklearn.datasets import make_classification
from scipy.stats import sem
from scipy import stats
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests
from yellowbrick.regressor import PredictionError, ResidualsPlot
from bioinfokit import analys, visuz

### Code to import and generate datasets all analysis is using.

In [ ]:
# Import all datasets.

# Covid donors cell matrices
covid_B = pd.read_csv("./all_covid_B_cells.csv")
covid_CD8 = pd.read_csv("./all_covid_CD8_cells.csv")

# Healthy donors cell matrices
healthy_B = pd.read_csv("./all_healthy_B_cells.csv")
healthy_CD8 = pd.read_csv("./all_healthy_CD8_cells.csv")

# Reset index to donor's cells
covid_B = covid_B.rename(columns={'Unnamed: 0': 'Donor_Cells'})
covid_B = covid_B.set_index('Donor_Cells')

covid_CD8 = covid_CD8.rename(columns={'Unnamed: 0': 'Donor_Cells'})
covid_CD8 = covid_CD8.set_index('Donor_Cells')

healthy_B = healthy_B.rename(columns={'Unnamed: 0': 'Donor_Cells'})
healthy_B = healthy_B.set_index('Donor_Cells')

healthy_CD8 = healthy_CD8.rename(columns={'Unnamed: 0': 'Donor_Cells'})
healthy_CD8 = healthy_CD8.set_index('Donor_Cells')

Inserting dummy variable to represent COVID status - 1 represents COVID positive donor, 0 represents healthy donor.

In [ ]:
# Add dummy variable to datasets, before combining.
covid_B_dummy = [1] * len(covid_B)
covid_B.insert(1, 'Status', covid_B_dummy)

healthy_B_dummy = [0] * len(healthy_B)
healthy_B.insert(1, 'Status', healthy_B_dummy)

covid_CD8_dummy = [1] * len(covid_CD8)
covid_CD8.insert(1, 'Status', covid_CD8_dummy)

healthy_CD8_dummy = [0] * len(healthy_CD8)
healthy_CD8.insert(1, 'Status', healthy_CD8_dummy)

In [ ]:
# Create combined datasets for both cell types with dummy variable included.
combined_B = covid_B.append(healthy_B)
combined_CD8 = covid_CD8.append(healthy_CD8)

In [ ]:
# Preview of combined datasets created:
print(combined_B.head())
print(combined_CD8.head())

### Exploratory Data Analysis

Arrays which contain how many cells each individual gene is expressed in. For example, value_count_cB[0] would produce how many cells the first gene, '5S-rRNA', in the gene expression matrix is expressed in for the covid B cells dataset.

In [ ]:
# Covid B cells count.
value_count_cB = [(covid_B[col] != 0).sum() for col in covid_B]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_cB = value_count_cB[2:]

# Healthy B cells count.
value_count_hB = [(healthy_B[col] != 0).sum() for col in healthy_B]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_hB = value_count_hB[2:]

# Combined COVID and Healthy B cells count.
value_count_B = [(combined_B[col] != 0).sum() for col in combined_B]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_B = value_count_B[2:]

# COVID CD8+ T cells count.
value_count_cCD8 = [(covid_CD8[col] != 0).sum() for col in covid_CD8]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_cCD8 = value_count_cCD8[2:]

# Healthy CD8+ T cells count.
value_count_hCD8 = [(healthy_CD8[col] != 0).sum() for col in healthy_CD8]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_hCD8 = value_count_hCD8[2:]

# Combined COVID and Healthy CD8+ T cells count.
value_count_CD8 = [(combined_CD8[col] != 0).sum() for col in combined_CD8]
# Remove first two columns, which are 'Patient' and 'Status'.
value_count_CD8 = value_count_CD8[2:]

Arrays which contain the number of genes expressed in _n_ cells for each dataset, where _n_ = 0, ... , _t_ and _t_ is the total number of cells in that dataset. For example, value_count_cB_overall[0] would produce how many genes are expressed in 0 cells in the covid B cells dataset.

In [ ]:
# Covid B cells gene count.
value_count_cB_overall = []
counter = 0
while (counter <= covid_B.shape[0]):
    number_of_cells = value_count_cB.count(counter)
    value_count_cB_overall.append(number_of_cells)
    counter = counter + 1

# Healthy B cells gene count.
value_count_hB_overall = []
counter = 0
while (counter <= healthy_B.shape[0]):
    number_of_cells = value_count_hB.count(counter)
    value_count_hB_overall.append(number_of_cells)
    counter = counter + 1
    
# Combined B cells gene count.
value_count_B_overall = []
counter = 0
while (counter <= combined_B.shape[0]):
    number_of_cells = value_count_B.count(counter)
    value_count_B_overall.append(number_of_cells)
    counter = counter + 1
    
# Covid CD8+ T cells gene count.
value_count_cCD8_overall = []
counter = 0
while (counter <= covid_CD8.shape[0]):
    number_of_cells = value_count_cCD8.count(counter)
    value_count_cCD8_overall.append(number_of_cells)
    counter = counter + 1
    
# Healthy CD8+ T cells gene count.
value_count_hCD8_overall = []
counter = 0
while (counter <= healthy_CD8.shape[0]):
    number_of_cells = value_count_hCD8.count(counter)
    value_count_hCD8_overall.append(number_of_cells)
    counter = counter + 1
    
# Combined CD8+ T cells gene count.
value_count_CD8_overall = []
counter = 0
while (counter <= combined_CD8.shape[0]):
    number_of_cells = value_count_CD8.count(counter)
    value_count_CD8_overall.append(number_of_cells)
    counter = counter + 1

Line graph visualising the number of genes expressed in less than 1% of cells for each dataset

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(20,15))

# Set maximum range of x-axis to 1% of total number of cells.
max_range_cB = int(len(covid_B)/100)
max_range_hB = int(len(healthy_B)/100)
max_range_B = int(len(combined_B)/100)
max_range_cCD8 = int(len(covid_CD8)/100)
max_range_hCD8 = int(len(healthy_CD8)/100)
max_range_CD8 = int(len(combined_CD8)/100)

# Plot each dataset's gene count.
axs[0, 0].plot(value_count_cB_overall[:max_range_cB+1])
axs[0, 0].set_title(label='Covid B gene count', fontsize=15)
axs[0, 1].plot(value_count_hB_overall[:max_range_hB+1])
axs[0, 1].set_xticks([0, 5, 10, 15, 20])
axs[0, 1].set_title(label='Healthy B gene count', fontsize=15)
axs[0, 2].plot(value_count_B_overall[:max_range_B+1])
axs[0, 2].set_title(label='Combined B gene count', fontsize=15)
axs[1, 0].plot(value_count_cCD8_overall[:max_range_cCD8+1])
axs[1, 0].set_title(label='Covid CD8+ T gene count', fontsize=15)
axs[1, 1].plot(value_count_hCD8_overall[:max_range_hCD8+1])
axs[1, 1].set_title(label='Healthy CD8+ T gene count', fontsize=15)
axs[1, 2].plot(value_count_CD8_overall[:max_range_CD8+1])
axs[1, 2].set_title(label='Combined CD8+ T gene count', fontsize=15)

for ax in axs.flat:
    ax.set(xlabel='Number of cells', ylabel='Number of genes',)

plt.show()

Arrays which contain the gene reads for cells for both cell types. For example, cell_reads_B[500] would return the number of cells from the combined B cells dataset which had 500 genes expressed in them.

In [ ]:
# B cells gene reads.
combined_B_transposed = combined_B.iloc[:,2:].T
cell_reads_B = [(combined_B_transposed[row] != 0).sum() for row in combined_B_transposed]

# CD8+ T cells gene reads.
combined_CD8_transposed = combined_CD8.iloc[:,2:].T
cell_reads_CD8 = [(combined_CD8_transposed[row] != 0).sum() for row in combined_CD8_transposed]

Histogram visualising the gene reads of cells for both datasets.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(20,10))

axs[0].hist(cell_reads_B, bins=1000)
axs[0].set_title(label='Cell reads in B cells', fontsize=15)
axs[1].hist(cell_reads_CD8, bins=1000)
axs[1].set_title(label='Cell reads in CD8+ T cells', fontsize=15)

for ax in axs.flat:
    ax.set(xlabel='Number of reads', ylabel='Number of cells',)

plt.show()

Testing different thresholds to filter out genes expressed in a small percentage of cells.

In [ ]:
# Count of how many genes would be removed at different threshholds (0.001 to 0.01 in increments of 0.001).
percent = 0.001
while (percent < 0.01):
    counter = 0
    for col in combined_B:
        # Ignore columns which are not genes.
        if (col == 'Patient' or col == 'Status'):
            continue
        # Add to counter if gene expression falls under threshold.
        if ((combined_B[col] != 0).sum()/len(combined_B) < percent):
            counter = counter + 1
    print("Total number of genes expressed in less than "+str(round(percent*100,1))+"% of B cells:"+str(counter))
    percent = percent + 0.001

percent = 0.001
while (percent < 0.01):
    counter = 0
    for col in combined_CD8:
        # Ignore columns which are not genes.
        if (col == 'Patient' or col == 'Status'):
            continue
        # Add to counter if gene expression falls under threshold.
        if ((combined_CD8[col] != 0).sum()/len(combined_CD8) < percent):
            counter = counter + 1
    print("Total number of genes expressed in less than "+ str(round(percent*100,1)) +"% of CD8+ T cells:"+str(counter))
    percent = percent + 0.001

In [ ]:
# Count of how many genes would be removed at different threshholds (0.01 to 0.1, or 1% to 10%, in increments of 0.01).
percent = 0.01
while (percent <= 0.1):
    counter = 0
    for col in combined_B:
        # Ignore columns which are not genes.
        if (col == 'Patient' or col == 'Status'):
            continue
        # Add to counter if gene expression falls under threshold.
        if ((combined_B[col] != 0).sum()/len(combined_B) < percent):
            counter = counter + 1
    print("Total number of genes expressed in less than "+str(int(percent*100))+"% of B cells:"+str(counter))
    # Increase threshold by chosen increment.
    percent = percent + 0.01

percent = 0.01
while (percent <= 0.1):
    counter = 0
    for col in combined_CD8:
        # Ignore columns which are not genes.
        if (col == 'Patient' or col == 'Status'):
            continue
        # Add to counter if gene expression falls under threshold.
        if ((combined_CD8[col] != 0).sum()/len(combined_CD8) < percent):
            counter = counter + 1
    print("Total number of genes expressed in less than "+ str(int(percent*100)) +"% of CD8+ T cells:"+str(counter))
    percent = percent + 0.01

Array containing genes which will be removed due to being present in less than 1% of cells.

In [ ]:
B_genes_removed = []
for col in combined_B:
    if (col == 'Patient' or col == 'Status'):
        continue
    if ((combined_B[col] != 0).sum()/len(combined_B) < 0.01):
        B_genes_removed.append(col)
print("Total number of genes expressed in less than 1% of B cells:"+str(len(B_genes_removed)))

CD8_genes_removed = []
for col in combined_CD8:
    if (col == 'Patient' or col == 'Status'):
        continue
    if ((combined_CD8[col] != 0).sum()/len(combined_CD8) < 0.01):
        CD8_genes_removed.append(col)
print("Total number of genes expressed in less than 1% of CD8+ T cells:"+str(len(CD8_genes_removed)))

Creating datasets with genes present in less than 1% of cells filtered out.

In [ ]:
combined_B_filtered = combined_B.drop(B_genes_removed, axis = 1)
combined_CD8_filtered = combined_CD8.drop(CD8_genes_removed, axis = 1)
covid_B_filtered = covid_B.drop(B_genes_removed, axis = 1)
healthy_B_filtered = healthy_B.drop(B_genes_removed, axis = 1)
covid_CD8_filtered = covid_CD8.drop(CD8_genes_removed, axis = 1)
healthy_CD8_filtered = healthy_CD8.drop(CD8_genes_removed, axis = 1)

### Dimensionality Reduction - UMAP

Courtesy to: https://chanzuckerberg.github.io/scRNA-python-workshop

##### CODE FOR B CELLS:

Creating expression matrix:

In [ ]:
# Note that this expression matrix will contain the original cell count.
bcells = combined_B_filtered.loc[:, ~combined_B_filtered.columns.isin(['Patient', 'Status'])]
bcells_count = bcells.transform(func = lambda x : 10 ** x - 1)

Creating metadata:

In [ ]:
metadata = combined_B_filtered[['Patient','Status']]
metadata_df = pd.DataFrame(metadata, columns=['Patient','Status'])
print(metadata_df)

In [ ]:
# Creating an AnnData object
adata = sc.AnnData(X = bcells_count, obs = metadata_df)
adata.write('./bcells_raw.h5ad') # the h5ad extension is AnnData-specific

Normalising gene expression:

In [ ]:
adata = sc.read('./bcells_raw.h5ad')

# The simplest way to normalize this data is to convert it to counts per million (CPM)  
adata_cpm = adata.copy() # apply this to a copy so we can compare methods
adata_cpm.raw = adata_cpm # store a copy of the raw values before normalizing
sc.pp.normalize_per_cell(adata_cpm, counts_per_cell_after=1e6)

# We used this method to normalise our data
sc.pp.log1p(adata_cpm)
sc.pp.scale(adata_cpm)

adata_cpm.write('./bcells_normalized.h5ad')

Choosing UMAP parameters (for min_dist and n_components):

In [ ]:
adata = sc.read('./bcells_normalized.h5ad')
sc.pp.neighbors(adata) # UMAP is based on the neighbor graph; we'll compute this first
sc.tl.umap(adata, min_dist=0.1,spread = 1.1, random_state=1, n_components=20)
sc.pl.umap(adata, color='Status')

In [ ]:
# Compare different plots to see which parameter values are the best
umap2 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=2, copy=True)
umap10 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=10, copy=True)
umap15 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=15, copy=True)
umap20 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=20, copy=True)
umap30 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=30, copy=True)

# When min distance is 0.1
fig, axs = plt.subplots(1, 5,figsize=(30,5),constrained_layout=True)
sc.pl.umap(umap2, color='Status', title="UMAP2", show=False, ax=axs[0])
sc.pl.umap(umap10, color='Status', title="UMAP10", show=False, ax=axs[1])
sc.pl.umap(umap15, color='Status', title="UMAP15", show=False, ax=axs[2])
sc.pl.umap(umap20, color='Status', title="UMAP20", show=False, ax=axs[3])
sc.pl.umap(umap30, color='Status',  title="UMAP30", show=False, ax = axs[4]) 

In [ ]:
adata.write('./bcells_embeddings.h5ad')
print(adata.obsm['X_umap']) # UMAP co-ordinates

Clustering plots generated using k-means clustering:

In [ ]:
adata = sc.read('./bcells_embeddings.h5ad')

# K-means clustering:
umap_coordinates = adata.obsm['X_umap'] # extract the UMAP coordinates for each cell
kmeans = KMeans(n_clusters=4, random_state=0).fit(umap_coordinates) # fix the random state for reproducibility

adata.obs['kmeans'] = kmeans.labels_ # retrieve the labels and add them as a metadata column in our AnnData object
adata.obs['kmeans'] = adata.obs['kmeans'].astype(str)

sc.pl.umap(adata, color='kmeans') # plot the results

Clustering plots generated using graph-based louvain clustering method:

In [ ]:
sc.tl.louvain(adata, resolution=0.1)
sc.pl.umap(adata, color='Status')
rand_index = adjusted_rand_score(adata.obs['Status'], adata.obs['louvain'])

In [ ]:
sc.tl.louvain(adata, resolution=0.1)
sc.pl.umap(adata, color='louvain')

In [ ]:
adata.write('./bcells_clusters.h5ad')

Creating dataframes for clusters:

In [ ]:
adata = sc.read('./bcells_clusters.h5ad')
# Raw counts
raw = pd.DataFrame(data=adata.raw.X, index=adata.raw.obs_names, columns=adata.raw.var_names)

# Metadata - print Status, Patient and Louvain columns
metadata = adata.obs

# merge datasets
raw.reset_index(drop=True, inplace=True)
metadata.reset_index(drop=True, inplace=True)
combined_bcells = raw.join(metadata.iloc[:,[0,1,4]])

# Export datasets for clusters 0, 1 and 2
cluster0_results = combined_bcells[combined_bcells.louvain == '0']
print(cluster0_results)
cluster0_results.to_csv('./cluster0_bcells.csv', index = False, header=True)

cluster1_results = combined_bcells[combined_bcells.louvain == '1']
print(cluster1_results)
cluster1_results.to_csv('./cluster1_bcells.csv', index = False, header=True)

cluster2_results = combined_bcells[combined_bcells.louvain == '2']
print(cluster2_results)
cluster2_results.to_csv('./cluster2_bcells.csv', index = False, header=True)

##### CODE FOR CD8+ T CELLS:

Creating expression matrix:

In [ ]:
# Note that this expression matrix will contain the original cell count.
cd8_cells = combined_CD8_filtered.loc[:, ~combined_CD8_filtered.columns.isin(['Patient', 'Status'])]
cd8_cells_count = cd8_cells.transform(func = lambda x : 10 ** x - 1)

Creating metadata:

In [ ]:
metadata = combined_CD8_filtered[['Patient','Status']]
metadata_df = pd.DataFrame(metadata, columns=['Patient','Status'])
print(metadata_df)

In [ ]:
# Creating an AnnData object
adata = sc.AnnData(X = cd8_cells_count, obs = metadata_df)
adata.write('./cd8cells_raw.h5ad') # the h5ad extension is AnnData-specific

Normalising gene expression:

In [ ]:
adata = sc.read('./cd8cells_raw.h5ad')

# The simplest way to normalize this data is to convert it to counts per million (CPM)  
adata_cpm = adata.copy() # apply this to a copy so we can compare methods
adata_cpm.raw = adata_cpm # store a copy of the raw values before normalizing
sc.pp.normalize_per_cell(adata_cpm, counts_per_cell_after=1e6)
# We used this method to normalise our data
sc.pp.log1p(adata_cpm)
sc.pp.scale(adata_cpm)

adata_cpm.write('./cd8cells_normalized.h5ad')

Choosing UMAP parameters (for min_dist and n_components):

In [ ]:
adata = sc.read('./cd8cells_normalized.h5ad')
sc.pp.neighbors(adata) # UMAP is based on the neighbor graph; we'll compute this first
sc.tl.umap(adata, min_dist=0.1,spread = 1.1, random_state=1, n_components=30)
sc.pl.umap(adata, color='Status')

In [ ]:
# Compare different plots to see which parameter values are the best
umap2 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=2, copy=True)
umap10 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=10, copy=True)
umap15 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=15, copy=True)
umap20 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=20, copy=True)
umap30 = sc.tl.umap(adata,min_dist=0.1,spread=1.1, n_components=30, copy=True)

# When min distance is 0.1
fig, axs = plt.subplots(1, 5,figsize=(30,5),constrained_layout=True)
sc.pl.umap(umap2, color='Status', title="UMAP2", show=False, ax=axs[0])
sc.pl.umap(umap10, color='Status', title="UMAP10", show=False, ax=axs[1])
sc.pl.umap(umap15, color='Status', title="UMAP15", show=False, ax=axs[2])
sc.pl.umap(umap20, color='Status', title="UMAP20", show=False, ax=axs[3])
sc.pl.umap(umap30, color='Status',  title="UMAP30", show=False, ax = axs[4]) 

In [ ]:
adata.write('./cd8cells_embeddings.h5ad')
print(adata.obsm['X_umap']) # UMAP co-ordinates

Clustering plots generated using k-means clustering:

In [ ]:
adata = sc.read('./cd8cells_embeddings.h5ad')

# K-means clustering:
umap_coordinates = adata.obsm['X_umap'] # extract the UMAP coordinates for each cell
kmeans = KMeans(n_clusters=4, random_state=0).fit(umap_coordinates) # fix the random state for reproducibility

adata.obs['kmeans'] = kmeans.labels_ # retrieve the labels and add them as a metadata column in our AnnData object
adata.obs['kmeans'] = adata.obs['kmeans'].astype(str)

sc.pl.umap(adata, color='kmeans') # plot the results

Clustering plots generated using graph-based louvain clustering method:

In [ ]:
sc.tl.louvain(adata, resolution=0.1)
sc.pl.umap(adata, color='Status')
rand_index = adjusted_rand_score(adata.obs['Status'], adata.obs['louvain'])

In [ ]:
sc.tl.louvain(adata, resolution=0.1)
sc.pl.umap(adata, color='louvain')

In [ ]:
adata.write('./cd8cells_clusters.h5ad')

Creating dataframes for clusters:

In [ ]:
adata = sc.read('./cd8cells_clusters.h5ad')

# Raw counts 
raw = pd.DataFrame(data=adata.raw.X, index=adata.raw.obs_names, columns=adata.raw.var_names)

# Metadata - print Status, Patient and Louvain columns
metadata = adata.obs
print(metadata.iloc[:,[0,1,4]])

# merge datasets
raw.reset_index(drop=True, inplace=True)
metadata.reset_index(drop=True, inplace=True)
combined_cd8cells = raw.join(metadata.iloc[:,[0,1,4]])
print(combined_cd8cells)

# Export datasets for clusters 0 and 1
cluster0_results = combined_cd8cells[combined_cd8cells.louvain == '0']
print(cluster0_results)
cluster0_results.to_csv('./cluster0_cd8cells.csv', index = False, header=True)

cluster1_results = combined_cd8cells[combined_cd8cells.louvain == '1']
print(cluster1_results)
cluster1_results.to_csv('./cluster1_cd8cells.csv', index = False, header=True)

### Volcano Plots

#### Volcano plots created on data prior to UMAP filtering:

##### CODE FOR B CELLS:

Transforming data back to original counts:

In [ ]:
total_b_original = combined_B.transform(func = lambda x : 10 ** x - 1)

Calculating p-values on original count data:

In [ ]:
pvalues = stats.ttest_ind(total_b_original.iloc[:3028,:],total_b_original.iloc[3028:,:],equal_var=False)[1]
# Remove 'Patient' and 'Status' column p-values
pvalues1 = pvalues[2:]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10267
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues)
pvalues = len(total_b_original)
total_b_original.loc[pvalues] = to_append

Calculating log fold change:

In [ ]:
covid_mean = total_b_original.iloc[:3028,:].mean()
healthy_mean = total_b_original.iloc[3028:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)

to_append = list(fold_change)
fold_change = len(total_b_original)
total_b_original.loc[fold_change] = to_append

Finding most significant genes using various filters:

In [ ]:
total_b_transposed = total_b_original.T
total_b_transposed = total_b_transposed.iloc[2:,:]
total_b_transposed['pvalues'] = total_b_transposed.iloc[:,-2]
total_b_transposed['log2FC'] = total_b_transposed.iloc[:,-2]
total_b_transposed['GeneNames'] = total_b_transposed.index

# Counting how many were significant
print("Total number of significant genes: "+str(total_b_transposed[total_b_transposed['pvalues'] < 0.05].shape[0]))

# Filter by p-value only, top 10 most signficant genes
print("P-value only filtered:")
print(total_b_transposed.nsmallest(10,['pvalues']))

# Filtering by |log2FC| >= 1
logFC_gt1_b = total_b_transposed[abs(total_b_transposed['log2FC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1_b.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plots:

In [ ]:
# P-value only filtered plot (labelled)
visuz.gene_exp.volcano(df=total_b_transposed, lfc='log2FC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, geneid="GeneNames",
    genenames=({"MT-CO1":"MT-CO1",
                "HLA-DPB1":"HLA-DPB1",
                "CD74":"CD74",
                "HLA-DRA":"HLA-DRA",
                "RPL13A":"RPL13A",
                "HLA-DQB1":"HLA-DQB1",
                "GLTSCR2":"GLTSCR2",
                "RPS4X":"RPS4X",
                "RPL3":"RPL3",
                "RPS3":"RPS3"}), show=True)

# P-value and log2 fold change filtered plot (labelled)
visuz.gene_exp.volcano(df=total_b_transposed, lfc='log2FC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, geneid="GeneNames",
    genenames=({"CD74":"CD74",
                 "HLA-DPB1":"HLA-DPB1",
                 "HLA-DRA":"HLA-DRA",
                 "GLTSCR2":"GLTSCR2",
                 "IFI27":"IFI27",
                 "MS4A1":"MS4A1",
                 "AC009501.4":"AC009501.4",
                 "HLA-DQB1":"HLA-DQB1",
                 "RPS3A":"RPS3A",
                 "RPL9":"RPL9",
                 "IFI6":"IFI6",
                 "HLA-DPA1":"HLA-DPA1",
                 "IGHG4":"IGHG4",
                 "MX1":"MX1",
                 "LTB":"LTB",
                 "CD79B":"CD79B",
                 "CD79A":"CD79A",
                 "XBP1":"XBP1",
                 "ELL2":"ELL2",
                 "CALR":"CALR"}), show=True)

##### CODE FOR CD8+ T CELLS:

Transforming data back to original counts:

In [ ]:
total_cd8_original = combined_CD8.transform(func = lambda x : 10 ** x - 1)

Calculating p-values on original count data:

In [ ]:
pvalues = stats.ttest_ind(total_cd8_original.iloc[:3877,:],total_cd8_original.iloc[3877:,:],equal_var=False)[1]
# Remove 'Patient' and 'Status' column p-values
pvalues1 = pvalues[2:]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10113
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues)
pvalues = len(total_b_original)
total_b_original.loc[pvalues] = to_append

Calculating log fold change:

In [ ]:
covid_mean = total_cd8_original.iloc[:3877,:].mean()
healthy_mean = total_cd8_original.iloc[3877:,:].mean()

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)

to_append = list(fold_change)
fold_change = len(total_cd8_original)
total_cd8_original.loc[fold_change] = to_append

Finding most significant genes using various filters:

In [ ]:
total_cd8_transposed = total_cd8_original.T
total_cd8_transposed = total_cd8_transposed.iloc[2:,:]
total_cd8_transposed['pvalues'] = total_cd8_transposed.iloc[:,-2]
total_cd8_transposed['log2FC'] = total_cd8_transposed.iloc[:,-2]
total_cd8_transposed['GeneNames'] = total_cd8_transposed.index

# Counting how many were significant
print("Total number of significant genes: "+str(total_cd8_transposed[total_cd8_transposed['pvalues'] < 0.05].shape[0]))

# Filter by p-value only, top 10 sigificant genes
print("P-value only filtered:")
print(total_cd8_transposed.nsmallest(10,['pvalues']))

# Filtering by |log2FC| >= 1
logFC_gt1_cd8 = total_cd8_transposed[abs(total_cd8_transposed['log2FC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1_cd8.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plots:

In [ ]:
# P-value only filtered plot (labelled)
visuz.gene_exp.volcano(df=total_cd8_transposed, lfc='log2FC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, geneid="GeneNames",
    genenames=({"MT-CO1":"MT-CO1",
                "MT-ND4":"MT-ND4",
                "MT-CYB":"MT-CYB",
                "MT-ND6":"MT-ND6",
                "MT-ND5":"MT-ND5",
                "MT-ATP6":"MT-ATP6",
                "MALAT1":"MALAT1",
                "MT-CO2":"MT-CO2",
                "RPS3":"RPS3",
                "RPS4X":"RPS4X"}))

# P-value and log2 fold change filtered plot (labelled)
visuz.gene_exp.volcano(df=total_cd8_transposed, lfc='log2FC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, geneid="GeneNames",
    genenames=({"MT-CO1":"MT-CO1",
                 "MT-ND4":"MT-ND4",
                 "MT-CYB":"MT-CYB",
                 "MT-ND6":"MT-ND6",
                 "MT-ND5":"MT-ND5",
                 "MT-ATP6":"MT-ATP6",
                 "XAF1":"XAF1",
                 "IFI44L":"IFI44L",
                 "MT-CO3":"MT-CO3",
                 "IFI44":"IFI44",
                 "MX1":"MX1",
                 "OAS3":"OAS3",
                 "PARP9":"PARP9",
                 "SUN2":"SUN2",
                 "EIF2AK2":"EIF2AK2",
                 "IFIT3":"IFIT3",
                 "EPSTI1":"EPSTI1",
                 "DTX3L":"DTX3L",
                 "IFI6":"IFI6",
                 "ISG15":"ISG15"}))

#### Volcano plots created on data post UMAP filtering:

##### CODE FOR B CELLS:

CLUSTER 0:

Transforming data back to original counts:

In [ ]:
cluster0_results = pd.read_csv("./cluster0_bcells.csv")
cluster0_results = cluster0_results.iloc[:,:-3].transform(lambda x: np.log10(x+1))

Calculating p-values on original count data:

In [ ]:
pvalues1 = stats.ttest_ind(cluster0_results.iloc[:1387,:],cluster0_results.iloc[1387:,:],equal_var=False)[1]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10267
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues1)
cluster0_results.loc["pvalues"] = to_append

Calculating log fold change:

In [ ]:
covid_mean = cluster0_results.iloc[:1387,:].mean()
healthy_mean = cluster0_results.iloc[1387:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)

to_append = list(fold_change)
cluster0_results.loc["logFC"] = to_append

Finding most significant genes using various filters:

In [ ]:
cluster0_results_transposed = cluster0_results.T
cluster0_results_transposed['GeneNames'] = cluster0_results_transposed.index
cluster0_results_transposed = cluster0_results_transposed[cluster0_results_transposed['pvalues'].notna()]

# Counting how many were significant
print("Total number of significant genes: "
      +str(cluster0_results_transposed[cluster0_results_transposed['pvalues'] < 0.05].shape[0]))

# Filter by p-value only, top 10 significant genes
print("P-value only filtered:")
print(cluster0_results_transposed.nsmallest(10,['pvalues']))

# Filtering by |log2FC| >= 1
logFC_gt1 = cluster0_results_transposed[abs(cluster0_results_transposed['logFC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plot:

In [ ]:
visuz.gene_exp.volcano(df=cluster0_results_transposed, lfc='logFC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, show=True)

CLUSTER 1:

Transforming data back to original counts:

In [ ]:
cluster1_results = pd.read_csv("./cluster1_bcells.csv")
cluster1_results = cluster1_results.iloc[:,:-3].transform(lambda x: np.log10(x+1))

Calculating p-values on original count data:

In [ ]:
pvalues1 = stats.ttest_ind(cluster1_results.iloc[:1344,:],cluster1_results.iloc[1344:,:],equal_var=False)[1]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10267
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues1)
cluster1_results.loc["pvalues"] = to_append

Calculating log fold change:

In [ ]:
covid_mean = cluster1_results.iloc[:1344,:].mean()
healthy_mean = cluster1_results.iloc[1344:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)

to_append = list(fold_change)
cluster1_results.loc["logFC"] = to_append

Finding most significant genes using various filters:

In [ ]:
cluster1_results_transposed = cluster1_results.T
cluster1_results_transposed['GeneNames'] = cluster1_results_transposed.index
cluster1_results_transposed = cluster1_results_transposed[cluster1_results_transposed['pvalues'].notna()]
# Retaining genes with |logFC| <= 10 
cluster1_results_transposed = cluster1_results_transposed[abs(cluster1_results_transposed['logFC']) <= 10]

# Counting how many were significant
print("Total number of significant genes: "
      +str(cluster1_results_transposed[cluster1_results_transposed['pvalues'] < 0.05].shape[0]))

# Filter by p-value only, top 10 signficant genes
print("P-value only filtered:")
print(cluster1_results_transposed.nsmallest(10,['pvalues']))

# Filtering by |log2FC| >= 1
logFC_gt1 = cluster1_results_transposed[abs(cluster1_results_transposed['logFC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plot:

In [ ]:
visuz.gene_exp.volcano(df=cluster1_results_transposed, lfc='logFC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, show=True)

CLUSTER 2:

Transforming data back to original counts:

In [ ]:
cluster2_results = pd.read_csv("./cluster2_bcells.csv")
cluster2_results = cluster2_results.iloc[:,:-3].transform(lambda x: np.log10(x+1))

Calculating p-values on original count data:

In [ ]:
pvalues1 = stats.ttest_ind(cluster2_results.iloc[:297,:],cluster2_results.iloc[297:,:],equal_var=False)[1]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10267
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues1)
cluster2_results.loc["pvalues"] = to_append

Calculating log fold change:

In [ ]:
covid_mean = cluster2_results.iloc[:297,:].mean()
healthy_mean = cluster2_results.iloc[297:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)
fold_change[fold_change == -inf] = 0

to_append = list(fold_change)
cluster2_results.loc["logFC"] = to_append

Finding most significant genes using various filters:

In [ ]:
cluster2_results_transposed = cluster2_results.T
cluster2_results_transposed['GeneNames'] = cluster2_results_transposed.index

# Counting how many were significant
print("Total number of significant genes: "
      +str(cluster2_results_transposed[cluster2_results_transposed['pvalues'] < 0.05].shape[0]))

# Filtering by |log2FC| >= 1
logFC_gt1 = cluster2_results_transposed[abs(cluster2_results_transposed['logFC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1.nsmallest(20,['pvalues']).iloc[:,-3:])

##### CODE FOR CD8+ T CELLS:

CLUSTER 0:

Transforming data back to original counts:

In [ ]:
cluster0_results = pd.read_csv("./cluster0_cd8cells.csv")
cluster0_results = cluster0_results.loc[:, (cluster0_results != 0).any(axis=0)]
cluster0_results = cluster0_results.iloc[:,:-3].transform(lambda x: np.log10(x+1))

Calculating p-values on original count data:

In [ ]:
pvalues1 = stats.ttest_ind(cluster0_results.iloc[:2617,:],cluster0_results.iloc[2617:,:],equal_var=False)[1]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10113
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues1)
cluster0_results.loc["pvalues"] = to_append

Calculating log fold change:

In [ ]:
covid_mean = cluster0_results.iloc[:2617,:].mean()
healthy_mean = cluster0_results.iloc[2617:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)
fold_change[fold_change == -inf] = 0

to_append = list(fold_change)
cluster0_results.loc["logFC"] = to_append

Finding most significant genes using various filters:

In [ ]:
cluster0_results_transposed = cluster0_results.T
cluster0_results_transposed['GeneNames'] = cluster0_results_transposed.index
cluster0_results_transposed = cluster0_results_transposed[cluster0_results_transposed['pvalues'].notna()]
# Retaining genes with |logFC| <= 10
cluster0_results_transposed = cluster0_results_transposed[abs(cluster0_results_transposed['logFC']) <= 10]

# Counting how many were significant
print("Total number of significant genes: "
      +str(cluster0_results_transposed[cluster0_results_transposed['pvalues'] < 0.05].shape[0]))

# Filtering by |log2FC| >= 1
logFC_gt1 = cluster0_results_transposed[abs(cluster0_results_transposed['logFC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plot:

In [ ]:
visuz.gene_exp.volcano(df=cluster0_results_transposed, lfc='logFC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, show=True)

CLUSTER 1:

Transforming data back to original counts:

In [ ]:
cluster1_results = pd.read_csv("./cluster1_cd8cells.csv")
cluster1_results = cluster1_results.iloc[:,:-3].transform(lambda x: np.log10(x+1))

Calculating p-values on original count data:

In [ ]:
pvalues1 = stats.ttest_ind(cluster1_results.iloc[:407,:],cluster1_results.iloc[407:,:],equal_var=False)[1]

Performing multiple testing correction:

In [ ]:
print(len(pvalues1[np.where(pvalues1<0.05)]))

# Bonferroni correction
# Get Bonferroni corrected P-value, which is 0.0001
bf_p = 0.05/10113
# count P-values < bf_p
# output: 0
print("After Bonferroni: "+str(len(pvalues1[np.where(pvalues1<bf_p)])))

# Benjamini and Hochberg FDR at alpha=0.05
y=multipletests(pvals=pvalues1, alpha=0.05, method="fdr_bh")
# output: 0
print("After Benjamini/Hochberg correction :"+str(len(y[1][np.where(y[1]<0.05)])))  # y[1] returns corrected P-vals (array)

In [ ]:
to_append = list(pvalues1)
cluster1_results.loc["pvalues"] = to_append

Calculating log fold change:

In [ ]:
covid_mean = cluster1_results.iloc[:407,:].mean()
healthy_mean = cluster1_results.iloc[407:,:].mean()
# To prevent undefined fold changes, any mean values = 0 were replaced with 0.0001
healthy_mean[healthy_mean == 0] = 0.0001

fold_change = (covid_mean/healthy_mean) 
fold_change = np.log2(fold_change)
fold_change[fold_change == -inf] = 0

to_append = list(fold_change)
cluster1_results.loc["logFC"] = to_append

Finding most significant genes using various filters:

In [ ]:
cluster1_results_transposed = cluster1_results.T
cluster1_results_transposed['GeneNames'] = cluster1_results_transposed.index
cluster1_results_transposed = cluster1_results_transposed[cluster1_results_transposed['pvalues'].notna()]
# Retaining genes with |logFC| <= 10
cluster1_results_transposed = cluster1_results_transposed[abs(cluster1_results_transposed['logFC']) <= 10]

# Counting how many were significant
print("Total number of significant genes: "
      +str(cluster1_results_transposed[cluster1_results_transposed['pvalues'] < 0.05].shape[0]))

# Filter by p-value only, top 10 signficant genes
print("P-value only filtered:")
print(cluster1_results_transposed.nsmallest(10,['pvalues']))

# Filtering by |log2FC| >= 1
logFC_gt1 = cluster1_results_transposed[abs(cluster1_results_transposed['logFC']) >= 1]
# Top 20 most significant genes
print("P-value and log2 fold change filtered:")
print(logFC_gt1.nsmallest(20,['pvalues']).iloc[:,-3:])

Visualising signficant genes using volcano plot:

In [ ]:
visuz.gene_exp.volcano(df=cluster1_results_transposed, lfc='logFC', pv='pvalues', plotlegend=True, legendpos='upper right', 
    legendanchor=(1.46,1),gstyle=2, sign_line=True, show=True)

### Conceptual Model - LASSO Regression

Summarising information to a patient level:

In [ ]:
mean_B = combined_B_filtered.groupby('Patient').mean()
mean_CD8 = combined_CD8_filtered.groupby('Patient').mean()

In [ ]:
mean_B.loc[1:7,'Status'] = 1
mean_CD8.loc[1:7,'Status'] = 1

##### CODE FOR B CELLS:

Extracting genes which were statistically significant from volcano plots for LASSO:

In [ ]:
B_0 = mean_B.loc[:, mean_B.columns.isin(['Status', 'HLA-DQB1', 'IFI44L', 'DSP', 'S100A8', 
                                         'XIST', 'S100A9', 'IFI44', 'IFIT3', 'EML6', 'PARP9', 
                                         'VCAN', 'ARHGAP24', 'OAS3', 'PPDPF', 'HBB', 'IFI27', 
                                         'RP5-887A10.1', 'STMN3', 'ALDH16A1', 'TEX9'])]
B_1 = mean_B.loc[:, mean_B.columns.isin(['Status', 'IFI27', 'CLU', 'CD3D', 'F13A1', 'HIST1H2AC',
                                         'ANAPC1', 'BZW2', 'IGHV3-9', 'FASTKD1', 'SIGLEC1', 
                                         'CHSY1', 'TCTN3', 'IGLV1-40', 'COLGALT1', 'MKKS', 
                                         'VWA8', 'ITGB3', 'PTP4A3', 'ZCCHC8', 'IGLV3-10'])]
B_2 = mean_B.loc[:, mean_B.columns.isin(['Status','IL32', 'IFI44L', 'MX1', 'DSP', 'S100A8', 
                                         'IGLC2', 'SPON2', 'PDIA4', 'CD300A', 'SOCS3', 'MATK',
                                         'IFI44', 'LINC00623', 'CHST2', 'TAPSAR1', 'PILRB', 
                                         'PPP1R12C', 'P4HA1', 'METRNL', 'MYOM2'])]

Creating training and test data for each cluster:

In [ ]:
X_B0 = B_0.drop(['Status'], axis=1)
y_B0 = B_0['Status']

X_B1 = B_1.drop(['Status'], axis=1)
y_B1 = B_1['Status']

X_B2 = B_2.drop(['Status'], axis=1)
y_B2 = B_2['Status']

Running LASSO regression on cluster 0 and computing and plotting leave one out cross validation scores for various L1 penalty coefficients:

In [ ]:
# Fitting LASSO regression to data
cs = l1_min_c(X_B0, y_B0, loss="log") * np.logspace(0, 5, 12)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

# Computing cross validation scores
coefs_ = []
cv_scores = []
cv = KFold(n_splits=13, random_state=1, shuffle=True)
for c in cs:
    scores=[]
    clf.set_params(C=c)
    clf.fit(X_B0, y_B0)
    coefs_.append(clf.coef_.ravel().copy())
    scores = (cross_val_score(clf, X_B0, y_B0, scoring='accuracy', cv=cv, n_jobs=-1))
    cv_scores.append(mean(scores))
    print(c, 'Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Plotting cross validation scores
cv_scores = np.array(cv_scores)
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), cv_scores, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Cross Validation Accuracy Score")
plt.title("Cross Validation Accuracy Scores for Logistic LASSO Models", 
          "with Various L1 Penalty Coefficients in UMAP Cluster #0 for B Cells")
plt.axis("tight")
plt.show()

Print and plotting L1 regularization path:

In [ ]:
print(coefs_)
plt.plot(np.log10(cs), coefs_, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Coefficients")
plt.title("L1 Regularization Path for UMAP Cluster #0 for B Cells")
plt.legend(X_B0.columns.values, bbox_to_anchor=(1, 1))
plt.axis("tight")
plt.show()

Running LASSO regression on cluster 1 and computing leave one out cross validation scores for various L1 penalty coefficients:

In [ ]:
# Fitting LASSO regression to data
cs = l1_min_c(X_B1, y_B1, loss="log") * np.logspace(0, 5, 12)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

# Computing cross validation scores
coefs_ = []
cv_scores = []
cv = KFold(n_splits=13, random_state=1, shuffle=True)
for c in cs:
    scores=[]
    clf.set_params(C=c)
    clf.fit(X_B1, y_B1)
    coefs_.append(clf.coef_.ravel().copy())
    scores = (cross_val_score(clf, X_B1, y_B1, scoring='accuracy', cv=cv, n_jobs=-1))
    cv_scores.append(mean(scores))
    print(c, 'Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Plotting cross validation scores
cv_scores = np.array(cv_scores)
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), cv_scores, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Cross Validation Accuracy Score")
plt.title("Cross Validation Accuracy Scores for Logistic LASSO Models with Various L1 Penalty Coefficients in UMAP Cluster #1 for B Cells")
plt.axis("tight")
plt.show()

Printing and plotting L1 regularization path:

In [ ]:
print(coefs_)
plt.plot(np.log10(cs), coefs_, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Coefficients")
plt.title("L1 Regularization Path for UMAP Cluster #1 for B Cells")
plt.legend(X_B1.columns.values, bbox_to_anchor=(1, 1))
plt.axis("tight")
plt.show()

Running LASSO regression on cluster 2 and computing leave one out cross validation scores for various L1 penalty coefficients:

In [ ]:
# Fitting LASSO regression to data
cs = l1_min_c(X_B2, y_B2, loss="log") * np.logspace(0, 5, 12)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

# Computing cross validation scores
coefs_ = []
cv_scores = []
cv = KFold(n_splits=13, random_state=1, shuffle=True)
for c in cs:
    scores=[]
    clf.set_params(C=c)
    clf.fit(X_B2, y_B2)
    coefs_.append(clf.coef_.ravel().copy())
    scores = (cross_val_score(clf, X_B2, y_B2, scoring='accuracy', cv=cv, n_jobs=-1))
    cv_scores.append(mean(scores))
    print(c, 'Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Plotting cross validation scores
cv_scores = np.array(cv_scores)
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), cv_scores, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Cross Validation Accuracy Score")
plt.title("Cross Validation Accuracy Scores for Logistic LASSO Models with Various L1 Penalty Coefficients in UMAP Cluster #2 for B Cells")
plt.axis("tight")
plt.show()

Printing and plotting L1 regularization path:

In [ ]:
print(coefs_)
plt.plot(np.log10(cs), coefs_, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Coefficients")
plt.title("L1 Regularization Path for UMAP Cluster #2 for B Cells")
plt.legend(X_B2.columns.values, bbox_to_anchor=(1, 1))
plt.axis("tight")
plt.show()

##### CODE FOR CD8+ T CELLS:

Extracting genes which were statistically significant from volcano plots for LASSO:

In [ ]:
CD8_0 = mean_CD8.loc[:, mean_CD8.columns.isin(['Status', 'MAPKAPK2', 'RP11-284N8.3', 'ZFAND5', 'SRSF9', 
                                               'CASP8AP2', 'IFI44', 'BRD7', 'HECTD4', 'ATXN2L', 'IPO5', 
                                               'IFIH1', 'GIMAP8', 'MON2', 'SLC15A4', 'ENDOD1', 'PAIP2', 
                                               'ANO6', 'ITPR1', 'NXF1', 'IGHG4'])]
CD8_1 = mean_CD8.loc[:, mean_CD8.columns.isin(['Status','XIST', 'IGLC3', 'PDZD4', 'ZNF683', 'RASSF1', 
                                               'S100A8', 'BCR', 'ZNRD1-AS1', 'HLA-DQB1', 'CMKLR1', 
                                               'PCNXL3', 'IGHG1', 'AGAP3', 'HCP5', 'NPC1', 'PILRB', 
                                               'AGAP1', 'MCM7', 'MED12', 'PLXND1'])]

Creating training and test data for each cluster:

In [ ]:
X_CD8_0 = CD8_0.drop(['Status'], axis=1)
y_CD8_0 = CD8_0['Status']

X_CD8_1 = CD8_1.drop(['Status'], axis=1)
y_CD8_1 = CD8_1['Status']

Running LASSO regression on cluster 0 and computing and plotting leave one out cross validation scores for various L1 penalty coefficients:

In [ ]:
# Fitting LASSO regression to data
cs = l1_min_c(X_CD8_0, y_CD8_0, loss="log") * np.logspace(0, 5, 12)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

# Computing cross validation scores
coefs_ = []
cv_scores = []
cv = KFold(n_splits=13, random_state=1, shuffle=True)
for c in cs:
    scores=[]
    clf.set_params(C=c)
    clf.fit(X_CD8_0, y_CD8_0)
    coefs_.append(clf.coef_.ravel().copy())
    scores = (cross_val_score(clf, X_CD8_0, y_CD8_0, scoring='accuracy', cv=cv, n_jobs=-1))
    cv_scores.append(mean(scores))
    print(c, 'Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Plotting cross validation scores
cv_scores = np.array(cv_scores)
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), cv_scores, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Cross Validation Accuracy Score")
plt.title("Cross Validation Accuracy Scores for Logistic LASSO Models with Various L1 Penalty Coefficients in UMAP Cluster #0 for CD8+ T Cells")
plt.axis("tight")
plt.show()

Printing and plotting L1 regularization path:

In [ ]:
print(coefs_)
plt.plot(np.log10(cs), coefs_, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Coefficients")
plt.title("L1 Regularization Path for UMAP Cluster #0 for CD8+ T Cells")
plt.legend(X_CD8_0.columns.values, bbox_to_anchor=(1, 1))
plt.axis("tight")
plt.show()

Running LASSO regression on cluster 1 and computing and plotting leave one out cross validation scores for various L1 penalty coefficients:

In [ ]:
# Fitting LASSO regression to data
cs = l1_min_c(X_CD8_1, y_CD8_1, loss="log") * np.logspace(0, 5, 12)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

# Computing cross validation scores
coefs_ = []
cv_scores = []
cv = KFold(n_splits=13, random_state=1, shuffle=True)
for c in cs:
    scores=[]
    clf.set_params(C=c)
    clf.fit(X_CD8_1, y_CD8_1)
    coefs_.append(clf.coef_.ravel().copy())
    scores = (cross_val_score(clf, X_CD8_1, y_CD8_1, scoring='accuracy', cv=cv, n_jobs=-1))
    cv_scores.append(mean(scores))
    print(c, 'Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

# Plotting cross validation scores
cv_scores = np.array(cv_scores)
coefs_ = np.array(coefs_)
plt.plot(np.log10(cs), cv_scores, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Cross Validation Accuracy Score")
plt.title("Cross Validation Accuracy Scores for Logistic LASSO Models with Various L1 Penalty Coefficients in UMAP Cluster #1 for CD8+ T Cells")
plt.axis("tight")
plt.show()

Printing and plotting L1 regularization path:

In [ ]:
print(coefs_)
plt.plot(np.log10(cs), coefs_, marker="o")
ymin, ymax = plt.ylim()
plt.xlabel("log(C)")
plt.ylabel("Coefficients")
plt.title("L1 Regularization Path for UMAP Cluster #1 for CD8+ T Cells")
plt.legend(X_CD8_1.columns.values, bbox_to_anchor=(1, 1))
plt.axis("tight")
plt.show()

---